In [1]:
import pandas as pd

In [3]:
# load data
df_complete = pd.read_csv("../data/wide_table.csv")
parameters = list(filter(lambda x: x.startswith('P_'), list(df_complete.columns)))
df = df_complete[["LATITUDE", "LONGITUDE", "LEV_M"] + parameters]